In [ ]:
import google.generativeai as genai
import requests
from bs4 import BeautifulSoup
import subprocess
import os

# Configure the Gemini client with your API key
genai.configure(api_key="")
model = genai.GenerativeModel("gemini-1.5-flash")

def fetch_jira_bug_report(jira_url):
    """
    Fetch and parse a bug report from an Apache Jira link, including the priority.
    :param jira_url: URL to the Jira issue.
    :return: Bug report details including priority as a string.
    """
    try:
        response = requests.get(jira_url)
        if response.status_code != 200:
            raise Exception(f"Failed to fetch Jira URL. HTTP Status Code: {response.status_code}")
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract bug report title
        title_element = soup.find("h1")
        title = title_element.text.strip() if title_element else "No title available."

        # Extract description
        description_element = soup.find("div", {"id": "descriptionmodule"})
        description = description_element.text.strip() if description_element else "No description available."

        # Extract priority
        priority_element = soup.find("span", {"id": "priority-val"})
        priority = priority_element.text.strip() if priority_element else "No priority available."

        # Combine title, description, and priority
        bug_report = (
            f"Title: {title}\n\n"
            f"Description: {description}\n\n"
            f"Priority: {priority}"
        )
        return bug_report
    except Exception as e:
        print(f"Error fetching Jira bug report: {e}")
        return ""

def fetch_code_version(project_path, version_id, exclude_tests=False):
    """
    Navigate to the project directory and fetch code for a specific version using 'git show'.
    Optionally excludes test-related changes from the output.
    :param project_path: Path to the Git project directory.
    :param version_id: The Git version ID.
    :param exclude_tests: If True, exclude test-related changes.
    :return: The code as a string.
    """
    original_path = os.getcwd()  # Save the current working directory
    try:
        # Navigate to the project directory
        os.chdir(project_path)
        
        # Run 'git show' command
        result = subprocess.run(
            ["git", "show", version_id],
            capture_output=True,
            text=True,
            check=True
        )
        code = result.stdout
        
        # Exclude test-related changes if required
        if exclude_tests:
            test_marker = "diff --git a/src/test/"
            code = code.split(test_marker, 1)[0]  # Remove everything starting from the marker
        
        return code
    except subprocess.CalledProcessError as e:
        print(f"Error fetching code for version {version_id}: {e}")
        return ""
    finally:
        # Change back to the original working directory
        os.chdir(original_path)

# Jira bug report link
jira_link = "https://issues.apache.org/jira/browse/LANG-693"

# Git project directory path and version IDs
project_path = "/mnt/seconddisk/master/courses/cs588/playgrounds.ipynb/commons-lang"  # Replace with the actual project path
buggy_version_id = "64cfee77e333d0c31a0fde0abe6dac3d97b0f078"  # Replace with the actual buggy version ID
fixed_version_id = "8a1042959df80c06dbfa83896594caa8e20ff9d6"  # Replace with the actual fixed version ID

# Fetch bug report details from Jira
bug_reports = fetch_jira_bug_report(jira_link)

# Fetch buggy and fixed code versions
buggy_code = fetch_code_version(project_path, buggy_version_id)
fixed_code = fetch_code_version(project_path, fixed_version_id, exclude_tests=True)

# User query (user_query) - Request for unit tests
user_query = "Generate unit tests based on the provided bug reports and code changes."

# Create a prompt combining bug reports, buggy code, and fixed code
if bug_reports.strip():
    prompt = (
        f"Use the following bug/error report and code versions to generate unit tests that address the issues:\n\n"
        f"Bug Report:\n{bug_reports}\n\n"
        f"Buggy Code:\n{buggy_code}\n\n"
        f"Fixed Code:\n{fixed_code}\n\n"
        f"Task: {user_query}"
    )
else:
    prompt = f"Task: {user_query}\n\nNote: No specific bug reports were provided. Generate generic unit tests based on your understanding."

# Generate a response with error handling
try:
    response = model.generate_content(prompt)
    print("Generated Unit Tests:\n")
    print(response.text)
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
print(fixed_code)